<a href="https://colab.research.google.com/github/lincgr/limpieza_datos/blob/master/Limpieza_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# Cargamos las librerias necesarias
import numpy as np
import pandas as pd

#Cargando datos adquiridos
#df = dataframe
df = pd.read_csv('/content/drive/MyDrive/Dataset/propiedades.csv')



In [23]:
#Mostrando una muestra de los datos. 
df.head()

,PROPIETARIO,METROS,PROPIEDAD_OCUPADA,NUM_CUARTOS
0,EDURNE,104.0,Y,3
1,DESIREE,197.0,N,3
2,DESIREE,NaN,N,NaN
3,ROSENDO,201.0,12,1
4,ROSENDO,203.0,Y,3


#Valores perdidos estándar
Entonces, ¿qué quiero decir con "valores perdidos estándar"? Estos son valores perdidos que Pandas puede detectar.

In [24]:
#Valores de la Calle
print(df['METROS'])


0    104.0
1    197.0
2      NaN
3    201.0
4    203.0
5    207.0
6      NaN
7    213.0
8    215.0
Name: METROS, dtype: float64


In [25]:
#Mostrar si son nulos o no
#Echando un vistazo a la columna, podemos ver que Pandas llenó el espacio en blanco con "NA". 
#Usando el método isnull (), podemos confirmar que tanto el valor perdido (Blanco) como “NA” fueron reconocidos como valores perdidos. 
#Ambas respuestas booleanas son verdaderas.

#Este es un ejemplo simple, pero destaca un punto importante. 
#Pandas reconocerá tanto las celdas vacías como los tipos "NA" como valores perdidos.


print(df['METROS'].isnull())

0    False
1    False
2     True
3    False
4    False
5    False
6     True
7    False
8    False
Name: METROS, dtype: bool


#Valores perdidos no estándar 
A veces puede darse el caso de que falten valores que tengan diferentes formatos. Por ejemplo en la columna Numero de cuartos [--, na]

In [26]:
df['NUM_CUARTOS']

0      3
1      3
2    NaN
3      1
4      3
5    NaN
6      2
7     --
8     na
Name: NUM_CUARTOS, dtype: object

In [27]:
df['NUM_CUARTOS'].isnull()

0    False
1    False
2     True
3    False
4    False
5     True
6    False
7    False
8    False
Name: NUM_CUARTOS, dtype: bool

Si varios usuarios ingresan datos manualmente, este es un problema común. Tal vez un usuario utiliza "--" pero otro usuario puede colocar "na".
Una forma sencilla de detectar estos distintos formatos es ponerlos en una lista.

na_values: se utiliza para crear una cadena que considera a los pandas como NaN (no es un número).


In [28]:
valores_faltants = ["na", "--"]
df = pd.read_csv('/content/drive/MyDrive/Dataset/propiedades.csv', na_values = valores_faltants)

In [29]:
print(df['NUM_CUARTOS'])
print(df['NUM_CUARTOS'].isnull())

0    3.0
1    3.0
2    NaN
3    1.0
4    3.0
5    NaN
6    2.0
7    NaN
8    NaN
Name: NUM_CUARTOS, dtype: float64
0    False
1    False
2     True
3    False
4    False
5     True
6    False
7     True
8     True
Name: NUM_CUARTOS, dtype: bool


#Valores perdidos inesperados
si se espera que nuestra función sea una cadena, pero hay un tipo numérico, técnicamente también es un valor faltante.

En la columna "PROPIEDAD_OCUPADA" tenemos un ejemplo de esto.

In [30]:
print(df['PROPIEDAD_OCUPADA'])
print(df['PROPIEDAD_OCUPADA'].isnull())

0      Y
1      N
2      N
3     12
4      Y
5      Y
6    NaN
7      Y
8      Y
Name: PROPIEDAD_OCUPADA, dtype: object
0    False
1    False
2    False
3    False
4    False
5    False
6     True
7    False
8    False
Name: PROPIEDAD_OCUPADA, dtype: bool


In [31]:
#En la cuarta fila, está el número 12. La respuesta para Propietario ocupado debe ser claramente una cadena (Y o N), por lo que este tipo numérico debe ser un valor faltante.
cnt=0
for row in df['PROPIEDAD_OCUPADA']:
    try:
        int(row)
        df.loc[cnt, 'PROPIEDAD_OCUPADA']=np.nan
    except ValueError:
        pass
    cnt+=1

En el código, recorremos cada entrada de la columna "PROPIEDAD_OCUPADA". Para intentar cambiar la entrada a un número entero, usamos int (fila).

Si el valor se puede cambiar a un número entero, cambiamos la entrada a un valor faltante usando np.nan de Numpy.
Por otro lado, si no se puede cambiar a un número entero, pasamos y seguimos.

El try y excepto ValueError, esto se llama manejo de excepciones y lo usamos para manejar errores.
Si intentáramos cambiar una entrada a un número entero y no se pudiera cambiar, se devolvería un ValueError y el código se detendría. Para lidiar con esto, usamos el manejo de excepciones para reconocer estos errores y continuar.
Otra parte importante del código es el método .loc. Este es el método de Pandas preferido para modificar entradas en su lugar, usando las coordenadas [fila, columna]

#Resumen de valores perdidos

Una vez que hayamos limpiado los valores faltantes, probablemente querremos resumirlos. Por ejemplo, podríamos querer mirar el número total de valores perdidos para cada característica.

In [32]:
#Numero de valores perdidos por cada característica
df.isnull().sum()

PROPIETARIO          0
METROS               2
PROPIEDAD_OCUPADA    2
NUM_CUARTOS          4
dtype: int64

In [33]:
#También es posible que deseemos obtener un recuento total de valores perdidos.
df.isnull().sum().sum()

8

#Reemplazo
A menudo, tendrá que averiguar cómo quiere manejar los valores perdidos.
A veces, simplemente querrá eliminar esas filas, otras veces las reemplazará.

In [34]:
#Reemplazar los valores perdidos con un número
df['METROS'].fillna(125, inplace=True)

In [35]:
df['METROS']

0    104.0
1    197.0
2    125.0
3    201.0
4    203.0
5    207.0
6    125.0
7    213.0
8    215.0
Name: METROS, dtype: float64

In [36]:
# Reemplazo basado en la ubicación
df.loc [2, 'METROS'] = 125

In [37]:
#Una forma muy común de reemplazar los valores perdidos es usar una mediana.
# Reemplazar usando mediana
mediana = df['NUM_CUARTOS'].median()
df['NUM_CUARTOS'].fillna(mediana, inplace=True)

In [38]:
df['NUM_CUARTOS']

0    3.0
1    3.0
2    3.0
3    1.0
4    3.0
5    3.0
6    2.0
7    3.0
8    3.0
Name: NUM_CUARTOS, dtype: float64

In [39]:
df.to_csv('/content/drive/MyDrive/Dataset/20210721_cleanData.csv',header=False)
